# Pricing a Snowball Option Using Monte Carlo Simulation and GPU Acceleration
## Introduction
- **Purpose of the Notebook**: To demonstrate how to price a Snowball option using Monte Carlo simulation, enhanced with GPU acceleration using Numba and CUDA.
- **Overview**:
  - Snowball Options: Explanation of what they are.
  - Monte Carlo Simulation: Importance in option pricing.
  - GPU Acceleration: How Numba and CUDA can speed up computations.
- **Note**:
  - I would assume you have prior knowledge on derivatives, what are they, the common types, nomenclatures, and their importance. If you need to catch up or a refresher, the book Option, Volatility and Pricing is a good place to start. I might write a primer later for reference.

Derivative: an financial asset derived from another financial asset.
Underlying: an financial asset that underlies the derivatives. Focus on the concept of underneath.

## 1. Snowball Option as a Barrier Option
- Definition:
  - Snowball option IS a type of barrier option.
  - Snowball option IS a type of autocallables.
  - Introduction of barrier option can be dated back to xxxx.
  - Snowball means it can roll bigger. The longer you hold the higher the payoff.
  - A common conspiracy is that the word "Snowball" sounds better than "barrier", hence the rephrase marketing.
  - You having the snowball product means youve sold a put option to the security institutions.
- Key Features:
  - Knock-In Barrier: A level that, when breached, activates certain payoff conditions.
    - Lower than the initial price
  - Knock-Out Barrier: A level that, when breached, terminates the option.
    - Slightly higher than the initial price
  - Observation Dates: When the Knock-in or Knock-out event is determined. Usually in terms of frequency.
- Payoff Structure:
  - Depends on whether the asset price hits the barriers during the option's life.
- Example:
  - Lets say there exist such product:


| Contract Term                   | Example        | Description |
|-------------------------------|------------|------------|
| Underlying                    |SPY | |
| Tenor                         |12 months ||
| Initial Observation Day |2024-09-13|The initial observation day is typically the start date or the next trading day|
| Initial/Reference Price |$559.09 |The closing price of the underlying asset on the initial observation day |
| Knock-out Observation Day     |Monthly| Fixed on a specific day each month. In this example would be 2024-10-13, 2024-11-13, etc...|
| Knock-in Observation Day      |Daily |Every trading day from the initial observation day to the final observation day. In this example would be 2024-09-13, 2024-09-14, etc...|
| Knock-out                     |103%|Knock-out occurs if the underlying price ≥ 103% * initial reference price|
| Knock-in                      |70%|Knock-in occurs if the underlying price < 70% * initial reference price|
| Coupon Rate                   |18%| (annualized); specific rate depends on knock-in or knock-out events |


收益计算规则：

1.发生敲出事件，产品提前结束，具体收益需要用年化票息率计算，比如第1个月就敲出，实际持有1个月，18%/12=1.5%，收益=本金*1.5%。

2.发生敲入事件，未发生敲出事件。如果到期日股价>期初价格，投资者收益为0；如果到期日股价<期初价格，需承担标的下跌损失。

3.如果敲入和敲出都没有发生，产品12个月到期，按约定票息率18%结算。

由于最后的收益取决于挂钩标的走势和敲入、敲出事件发生的时间，那么会出现几种情形呢？

在12个月期限内，共有5种可能性，逐一分析：

股价未下跌30%
1. 未敲入，提前敲出；  
2. 未敲入，未敲出；

股价曾出现跌幅超30%
3. 敲入，后敲出；
4. 敲入，未敲出，到期日股价≥期初价格；
5. 敲入，未敲出，到期日股价<期初价格；  

情景1：
观察期间，未发生敲入事件，即标的从未跌幅超过30%（标的收盘价<期初参考价*70%，敲入发生），假如在第5个月度观察日敲出（月度观察日一共有12个）
收益：18%/12*5=7.5%，如果本金100万，就是7.5万

情景2：

观察期间，未发生敲入事件，也未发生敲出事件。
收益：18%，12个月到期结束。如果本金100万，收益18万。

情景3：
观察期间，发生敲入事件，然后在第6个月触发敲出事件。
收益：18%/12*6=9.0%，如果本金100万，收益9万

情景4：

观察期间，发生敲入事件，未敲出，到期日股价>期初价格；
收益：0，本金依然为100万。

情景5：

观察期间，发生敲入事件，未敲出，到期日股价<期初价格，跌幅5%
发生亏损：亏损为挂钩标的实际跌幅，也就是5%，本金剩余95万，亏损5万

上面5种情景中，只有最后一种情况才会发生亏损的，整体是3赚1平1赔。上面的雪球结构表明，只要挂钩标的跌幅不超过30%，投资者不仅不用承担损失，相反，还能获得年化18%左右的收益。即使挂钩标的跌幅超过30%，也只有在到期日股价<期初价格时，才会产生亏损

2.1. Payoff Diagram
Visual representation of the payoff as a function of the underlying asset price.

3. Monte Carlo Simulation Basics <a id="section3"></a>
Purpose: To estimate the expected payoff by simulating numerous possible asset price paths.
Key Concepts:
Random Number Generation: Essential for simulating stochastic processes.
Time Steps: Discretization of the option's life.
Asset Price Dynamics: Usually modeled using Geometric Brownian Motion.

 3.1 Geometric Brownian Motion (GBM)
 equation: dS_t = miu * S_t *  dt + sigma * S_t & dW_t
 S_t: Aseet price at time t
 miu: drift term (expected return)
 sigma: volatility
 dW_t: Wiener process (standard Brownian motion)

4. Setting Up the Environment <a id="section4"></a>
Prerequisites:
Python 3.x
Libraries:
NumPy: Numerical computations.
Matplotlib: Visualization.
Numba: Just-in-time compilation to speed up Python code.
CUDA: GPU acceleration (requires compatible NVIDIA GPU).
Installation Instructions:
How to install each library using pip or conda.
Verify CUDA installation.

4.1. Importing Libraries

5. Simulating Asset Price Paths <a id="section5"></a>
Objective: Write code to simulate the asset price paths using Monte Carlo simulation.
Steps:
Define simulation parameters (initial price, drift, volatility, time steps).
Generate random shocks using standard normal distribution.
Update asset prices iteratively.

5.1. Basic Asset Simulation without GPU

def simulate_asset_prices(S0, mu, sigma, T, dt, N):
    """Simulate N asset price paths over time T with time step dt."""
    time_steps = int(T / dt)
    prices = np.zeros((N, time_steps))
    prices[:, 0] = S0
    for t in range(1, time_steps):
        z = np.random.standard_normal(N)
        prices[:, t] = prices[:, t-1] * np.exp((mu - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * z)
    return prices

5.2. Visualizing Sample Paths
# Parameters
S0 = 100
mu = 0.05
sigma = 0.2
T = 1.0
dt = 1/252  # Daily steps
N = 10  # Number of paths

# Simulate
prices = simulate_asset_prices(S0, mu, sigma, T, dt, N)

# Plot
plt.figure(figsize=(10, 6))
for i in range(N):
    plt.plot(prices[i])
plt.title('Simulated Asset Price Paths')
plt.xlabel('Time Steps')
plt.ylabel('Asset Price')
plt.show()

6. Implementing the Snowball Option Payoff <a id="section6"></a>
Objective: Calculate the payoff for each simulated path based on the Snowball option conditions.
Steps:
Initialize barrier flags for knock-in and knock-out.
Check barrier conditions at each time step.
Compute the final payoff based on the barriers and strike price.


6.1. Defining the Payoff Function
def calculate_payoff(prices, strike, knock_in, knock_out):
    """Calculate the payoff for each price path."""
    N = prices.shape[0]
    payoffs = np.zeros(N)
    for i in range(N):
        S_path = prices[i]
        barrier_knock_in = np.any(S_path <= knock_in)
        barrier_knock_out = np.any(S_path >= knock_out)
        S_T = S_path[-1]
        if barrier_knock_out:
            payoffs[i] = 0.0
        elif barrier_knock_in:
            payoffs[i] = max(S_T - strike, 0)
        else:
            payoffs[i] = 0.0
    return payoffs

6.2. Estimating the Option Price
# Parameters
strike = 100
knock_in = 90
knock_out = 110

# Calculate payoffs
payoffs = calculate_payoff(prices, strike, knock_in, knock_out)

# Estimate option price
option_price = np.mean(payoffs) * np.exp(-mu * T)
print(f"Estimated Snowball Option Price: {option_price:.4f}")

7. Enhancing Performance with Numba and CUDA <a id="section7"></a>
Objective: Accelerate the simulation using GPU parallelism.
Introduction to Numba and CUDA:
Numba: A JIT compiler that translates a subset of Python and NumPy code into fast machine code.
CUDA: A parallel computing platform and API model created by NVIDIA.

7.1. Setting Up GPU Functions
Random Number Generation on GPU:
Use xoroshiro128p generators from Numba's CUDA random module.
Defining the CUDA Kernel:
The kernel function will run on the GPU to simulate asset paths and compute payoffs.

@cuda.jit
def monte_carlo_snowball_kernel(result, dt, rate, volatility, strike, knock_in, knock_out, rng_states, time_steps):
    # Kernel code
    pass  # Detailed implementation goes here

7.2. Implementing the CUDA Kernel
Complete the Kernel Function:
Initialize asset price.
Loop over time steps, updating the asset price.
Check barrier conditions.
Calculate the payoff.
Store the result.

7.3. Launching the Kernel
Set Up Grid and Block Dimensions:

threads_per_block = 256
blocks_per_grid = (num_simulations + (threads_per_block - 1)) // threads_per_block

Create Random Number States:
rng_states = create_xoroshiro128p_states(num_simulations, seed=42)

Launch the Kernel:
monte_carlo_snowball_kernel[blocks_per_grid, threads_per_block](...)

8. Visualizing the Simulation Results <a id="section8"></a>
Objective: Provide real-time visualization of the running estimate and payoff distribution.
Tools:
Matplotlib for plotting.
plt.ion() for interactive mode.

8.1. Running Estimate Plot
Plot the estimated option price against the number of simulations completed.

8.2. Histogram of Payoffs
Plot the distribution of calculated payoffs.

8.3. Updating Plots in Real-Time
Use a loop to update the plots after each batch of simulations.
plt.ion()
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
# Loop to update plots
plt.ioff()

9. Full Application Code <a id="section9"></a>
Bringing Everything Together:
Combine all functions and code segments.
Provide the full code for the application.
Ensure the code is well-commented for clarity.

9.1. Main Execution Function
Define a main() function that sets parameters and calls other functions.

9.2. Running the Simulation
Execute the main() function.
Display the final estimated option price.

10. Conclusion <a id="section10"></a>
Summary:
Successfully priced a Snowball option using Monte Carlo simulation.
Enhanced performance using GPU acceleration.
Observations:
GPU acceleration significantly reduces computation time.
Visualization aids in understanding convergence and payoff distribution.
Potential Extensions:
Incorporate more complex option features.
Use quasi-random number generators for variance reduction.
Parallelize further using multiple GPUs or distributed computing.

11. References <a id="section11"></a>
Books, articles, and online resources for further reading.
Hull, J. C. Options, Futures, and Other Derivatives.
Numba and CUDA documentation.
[Articles on Snowball options and structured products.](https://xueqiu.com/9322843818/165295388)